# Class definitions

## Houseprint Singleton

The Houseprint is a Singleton object which contains all metadata for sites, devices and sensors. It can be pickled, saved and passed around

In [ ]:
class Houseprint(object):
    def __init__(self, spreadsheet = "Opengrid houseprint (Responses)"):
        """
            The Init method connects to the Google spreadsheet and parses the data into the OO structure.
        """
        
        """
            TODO: Google spreadsheet parsing
        """
        
        self.sites = []
        self.timestamp = None #Add a timestamp upon creation, so the server can check how old the houseprint info is and decide to request a new one.
    
    def get_sensors(self, sensortype = None):
        """
            Return a list with all sensors
            
            Parameters
            ----------
            sensortype: gas, water, electricity: optional
        """
        return [site.get_sensors(sensortype = sensortype) for site in self.sites]
    
    def get_devices(self):
        """
            Return a list with all devices
        """
        return [site.devices for site in self.sites]
    
    def save(self):
        """
        Pickle the houseprint object
        
        Parameters
        ----------
        * filename : str
            Filename, if relative path or just filename, it is appended to the
            current working directory
        
        """

        abspath = os.path.join(os.getcwd(), filename)
        f=file(abspath, 'w')
        pickle.dump(self, f)
        f.close()
        
        print("Saved houseprint to {}".format(abspath))
        
    def init_tmpo(self, tmpos=None):
        """
            Fluksosensors need a tmpo session to obtain data.
            It is overkill to have each fluksosensor make its own session, syncing would take too long and be overly redundant.
            Passing a tmpo session to the get_data function is also bad form because we might add new types of sensors that don't use tmpo in the future.
            This is why the session is initialised here.
            
            A tmpo session as parameter is optional.
            If no session is passed, a new one will be created using the location in the config file.
        """
        
        if tmpos is not None:
            self._tmpos = tmpos
        else:
            #get location from config file
            self._tmpos = tmpos#make new session
            
    def get_tmpos(self):
        """
            Returns
            -------
            TMPO session
        """
        #pseudocode
        if self.hasattr(_tmpos):
            return self._tmpos
        else:
            raise Error('No TMPO session was set, use the init_tmpo method to add or create a TMPO Session')
            
    def sync_tmpos(self):
        """
            Add all Fluksosensors to the TMPO session and sync
        """
        tmpos = self.get_tmpos()
        fluksosensors = [sensor for sensor in self.get_sensors() if sensor isInstanceOf(Fluksosensor)]
        
        for sensor in fluksosensors:
            tmpos.add(sensor.fluksoid, sensor.token)
            
        tmpos.sync()

## Site

A Site is a physical entity (a house, appartment, school, or other building). It may contain multiple devices and sensors.

The Site contains most of the metadata, eg. the number of inhabitants, the size of the building, the location etc.

In [ ]:
class Site(object):
    def __init__(self, hp, postcode = None, size = None, inhabitants = None):
        self.hp = hp #backref to parent
        self.postcode = postcode
        self.size = size
        self.inhabitants = inhabitants
        
        self.devices = []
        self.sensors = []
        
    def get_sensors(self, sensortype = None):
        """
            Return a list with all sensors
            
            Parameters
            ----------
            sensortype: gas, water, electricity: optional
        """
        return [sensor for sensor in self.sensors if sensor.type == sensortype or sensortype is None]

## Device

A Device is an entity that can contain multiple sensors.
The generic Device class can be inherited by a specific device class, eg. Fluksometer

In [ ]:
class Device(object):
    def __init__(self, site):
        self.site = site
        self.sensors = []
        
    def get_sensors(self, sensortype = None):
        """
            Return a list with all sensors
            
            Parameters
            ----------
            sensortype: gas, water, electricity: optional
        """
        return [sensor for sensor in self.sensors if sensor.type == sensortype or sensortype is None]
    
    def get_data(self, sensortype = None, head = None, tail = None):
        """
            Return a Pandas Dataframe with the joined data for all selected sensors
            
            Parameters
            ----------
            sensortype: gas, water, electricity: optional
            head, tail: timestamps indicating start and end
        """
        
        #pseudocode
        data = [sensor.get_data(head = head, tail = tail) for sensor in self.get_sensors(sensortype = sensortype)]
        return pd.concat(data) #or something...

In [ ]:
class Fluksometer(Device):
    def __init__(self, site, fluksoid, mastertoken = None):
        
        #invoke init method of generic Device
        super(Fluksometer, self).__init__(site)
        
        self.fluksoid = fluksoid
        self.mastertoken = mastertoken

## Sensor

A sensor generates a single data stream. It can have a parent device, but the possibility is also left open for a sensor to stand alone in a site.
It is an abstract class definition which has to be overridden (by eg. a Fluksosensor).

This class contains all metadata concerning the function and type of the sensor (eg. electricity - solar, ...)

In [ ]:
class Sensor(object):
    def __init__(self, site, type, function, unit, parent = None):
        self.site = site
        self.type = type
        self.function = function
        self.parent = parent
        self.unit = unit
        
    def get_data(self, head, tail):
        """
            Return a Pandas Series with measurement data
            
            Parameters
            ----------
            head, tail: timestamps for the begin and end of the interval
            
            Notes
            -----
            This is an abstract method, because each type of sensor has a different way of fetching the data.
        """
        
        raise NotImplementedError("Subclass must implement abstract method")

In [ ]:
class Fluksosensor(Sensor):
    def __init__(self, sensorid, parent, type, function, unit, token = None):
        
        #invoke init method of abstract Sensor
        super(Fluksosensor, self).__init__(site = parent.site,
                                           type = type,
                                           function = function,
                                           unit = unit,
                                           parent = parent)
        
        self.sensorid = sensorid
        if token is not None:
            self.token = token
        else:
            self.token = parent.mastertoken
        
    # @Override :-D    
    def get_data(self, head, tail):
        '''
            TODO: connect to TMPO and fetch data
        '''
        tmpos = self.site.hp.get_tmpos()
        
        data = tmpos.Series()
        
        return data